
# *7. Zaproponuj własną architekturę głębokiego Autoencodera wykorzystującego filtry konwolucyjne. Nowe podejście do ekstrakcji cech powinno poprawić dokładność klasyfikacji na wszystkich zbiorach danych.




DATA:

In [12]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(f_x_train, f_y_train), (f_x_test, f_y_test) = tf.keras.datasets.fashion_mnist.load_data()
k_x_train = np.load("kmnist-train-imgs.npz")['arr_0']
k_y_train = np.load("kmnist-train-labels.npz")['arr_0']
k_x_test = np.load("kmnist-test-imgs.npz")['arr_0']
k_y_test = np.load("kmnist-test-labels.npz")['arr_0']

In [13]:
y_train = np.array(y_train).astype(np.uint8)
y_test = np.array(y_test).astype(np.uint8)

f_y_train = np.array(f_y_train).astype(np.uint8)
f_y_test = np.array(f_y_test).astype(np.uint8)

k_y_train = np.array(k_y_train).astype(np.uint8)
k_y_test = np.array(k_y_test).astype(np.uint8)

In [14]:
x_train = x_train.astype(np.float32)/255.0
x_train = x_train.reshape(-1, 28, 28, 1)
f_x_train = f_x_train.astype(np.float32)/255.0
f_x_train = f_x_train.reshape(-1, 28, 28, 1)
k_x_train = k_x_train.astype(np.float32)/255.0
k_x_train = k_x_train.reshape(-1, 28, 28, 1)

x_test = x_test.astype(np.float32) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1)
f_x_test = f_x_test.astype(np.float32) / 255.0
f_x_test = f_x_test.reshape(-1, 28, 28, 1)
k_x_test = k_x_test.astype(np.float32) / 255.0
k_x_test = k_x_test.reshape(-1, 28, 28, 1)

In [15]:
print(x_train.shape)
print(x_test.shape)
print(f_x_train.shape)
print(f_x_test.shape)
print(k_x_train.shape)
print(k_x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 28, 28, 1)
(10000, 28, 28, 1)


Autoencoder:

In [50]:
class AutoencoderAdvanced(tf.keras.Model):
  def __init__(self, encoding_dimension, input_shape):
    super(AutoencoderAdvanced, self).__init__()
    self.encoding_dimension = encoding_dimension
    self.dropout_rate = 0.2
    self.leaky_rate = 0.1
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=input_shape),

            tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2)),
            tf.keras.layers.BatchNormalization(), 
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.LeakyReLU(negative_slope=self.leaky_rate),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2)),
            tf.keras.layers.BatchNormalization(), 
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.LeakyReLU(negative_slope=self.leaky_rate),

            tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=(2, 2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(self.dropout_rate), 
            tf.keras.layers.LeakyReLU(negative_slope=self.leaky_rate), 

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='leaky_relu'),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.Dense(256, activation='leaky_relu'),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.Dense(self.encoding_dimension),
        ]
    )
    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(self.encoding_dimension,)),

            tf.keras.layers.Dense(256, activation='leaky_relu'),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.Dense(512, activation='leaky_relu'),
            tf.keras.layers.Dropout(self.dropout_rate),

            tf.keras.layers.Dense(units=7*7*32, activation='leaky_relu'),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),


            tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.LeakyReLU(negative_slope=self.leaky_rate),

            tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.LeakyReLU(negative_slope=self.leaky_rate),

            tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.LeakyReLU(negative_slope=self.leaky_rate),

            tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', activation='sigmoid'),
        ]
    )

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

Computations:

In [51]:
input_shape = (28, 28, 1)
latent_dim = 196
batch_size = 32
epochs = 40

In [52]:
autoencoder_adv = AutoencoderAdvanced(latent_dim, input_shape)
autoencoder_adv.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=[tf.keras.metrics.Accuracy])

In [53]:
f_autoencoder_adv = AutoencoderAdvanced(latent_dim, input_shape)
f_autoencoder_adv.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=[tf.keras.metrics.Accuracy])

In [54]:
k_autoencoder_adv = AutoencoderAdvanced(latent_dim, input_shape)
k_autoencoder_adv.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=[tf.keras.metrics.Accuracy])

In [55]:
autoencoder_adv.fit(x_train, x_train,
                epochs=epochs,
                shuffle=True,
                batch_size=batch_size,
                validation_data=(x_test, x_test))

Epoch 1/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 141s 73ms/step - accuracy: 7.5776e-09 - loss: 0.1670 - val_accuracy: 1.2755e-07 - val_loss: 0.0866
Epoch 2/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 131s 70ms/step - accuracy: 2.3359e-08 - loss: 0.0863 - val_accuracy: 0.0000e+00 - val_loss: 0.0826
Epoch 3/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 117s 62ms/step - accuracy: 3.2392e-07 - loss: 0.0813 - val_accuracy: 7.6531e-07 - val_loss: 0.0812
Epoch 4/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 4.6676e-06 - loss: 0.0793 - val_accuracy: 1.9260e-05 - val_loss: 0.0786
Epoch 5/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 116s 62ms/step - accuracy: 5.6172e-05 - loss: 0.0778 - val_accuracy: 7.9209e-05 - val_loss: 0.0774
Epoch 6/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 116s 62ms/step - accuracy: 1.5759e-04 - loss: 0.0765 - val_accuracy: 1.7258e-04 - val_loss: 0.0766
Epoch 7/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 3.7182e-04 - loss: 0.0755 - val_accuracy: 9.1416e-04 - val_loss: 0.0757
Epoch 8/40
18

In [56]:
f_autoencoder_adv.fit(f_x_train, f_x_train,
                epochs=epochs,
                shuffle=True,
                batch_size=batch_size,
                validation_data=(f_x_test, f_x_test))

Epoch 1/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 123s 64ms/step - accuracy: 1.7779e-07 - loss: 0.3443 - val_accuracy: 0.0000e+00 - val_loss: 0.2941
Epoch 2/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 113s 60ms/step - accuracy: 1.4404e-07 - loss: 0.2893 - val_accuracy: 1.2755e-07 - val_loss: 0.2861
Epoch 3/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 113s 60ms/step - accuracy: 6.4162e-08 - loss: 0.2828 - val_accuracy: 0.0000e+00 - val_loss: 0.2825
Epoch 4/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 142s 60ms/step - accuracy: 1.2064e-07 - loss: 0.2797 - val_accuracy: 2.5510e-07 - val_loss: 0.2805
Epoch 5/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 114s 61ms/step - accuracy: 9.4011e-08 - loss: 0.2780 - val_accuracy: 1.2755e-07 - val_loss: 0.2804
Epoch 6/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 113s 61ms/step - accuracy: 1.8468e-07 - loss: 0.2767 - val_accuracy: 2.5510e-07 - val_loss: 0.2781
Epoch 7/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 142s 61ms/step - accuracy: 5.4094e-08 - loss: 0.2747 - val_accuracy: 0.0000e+00 - val_loss: 0.2779
Epoch 8/40
18

In [57]:
k_autoencoder_adv.fit(k_x_train, k_x_train,
                epochs=epochs,
                shuffle=True,
                batch_size=batch_size,
                validation_data=(k_x_test, k_x_test))

Epoch 1/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 116s 60ms/step - accuracy: 0.0000e+00 - loss: 0.3119 - val_accuracy: 1.2755e-07 - val_loss: 0.2176
Epoch 2/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 0.0000e+00 - loss: 0.1984 - val_accuracy: 0.0000e+00 - val_loss: 0.2023
Epoch 3/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 1.1416e-08 - loss: 0.1840 - val_accuracy: 0.0000e+00 - val_loss: 0.1937
Epoch 4/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 0.0000e+00 - loss: 0.1766 - val_accuracy: 0.0000e+00 - val_loss: 0.1887
Epoch 5/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 116s 62ms/step - accuracy: 2.0016e-08 - loss: 0.1725 - val_accuracy: 1.2755e-07 - val_loss: 0.1849
Epoch 6/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 5.2916e-08 - loss: 0.1688 - val_accuracy: 0.0000e+00 - val_loss: 0.1839
Epoch 7/40
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 111s 59ms/step - accuracy: 3.1330e-08 - loss: 0.1667 - val_accuracy: 0.0000e+00 - val_loss: 0.1809
Epoch 8/40
18

In [58]:
reconstructions_x_train = autoencoder_adv.predict(x_train)
reconstructions_x_train = reconstructions_x_train.reshape(-1, 28 * 28)
reconstructions_x_test = autoencoder_adv.predict(x_test)
reconstructions_x_test = reconstructions_x_test.reshape(-1, 28 * 28)

reconstructions_f_x_train = f_autoencoder_adv.predict(f_x_train)
reconstructions_f_x_train = reconstructions_f_x_train.reshape(-1, 28 * 28)
reconstructions_f_x_test = f_autoencoder_adv.predict(f_x_test)
reconstructions_f_x_test = reconstructions_f_x_test.reshape(-1, 28 * 28)

reconstructions_k_x_train = k_autoencoder_adv.predict(k_x_train)
reconstructions_k_x_train = reconstructions_k_x_train.reshape(-1, 28 * 28)
reconstructions_k_x_test = k_autoencoder_adv.predict(k_x_test)
reconstructions_k_x_test = reconstructions_k_x_test.reshape(-1, 28 * 28)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step


Classification:

In [59]:
x_train = x_train.reshape(-1, 784)
f_x_train = f_x_train.reshape(-1, 784)
k_x_train = k_x_train.reshape(-1, 784)

x_test = x_test.reshape(-1, 784)
f_x_test = f_x_test.reshape(-1, 784)
k_x_test = k_x_test.reshape(-1, 784)

In [60]:
classifier = LogisticRegression(solver='newton-cg')
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)
y_pred_recon = classifier.predict(reconstructions_x_test)
print(f"accuracy_score baseline: {accuracy_score(y_pred, y_test)}, accuracy_score reconstructions: {accuracy_score(y_pred_recon, y_test)}")

accuracy_score baseline: 0.9265, accuracy_score reconstructions: 0.929


In [66]:
f_classifier = LogisticRegression(solver='newton-cg')
f_classifier.fit(f_x_train, f_y_train)

y_pred = f_classifier.predict(f_x_test)
y_pred_recon = f_classifier.predict(reconstructions_f_x_test)
print(f"accuracy_score baseline: {accuracy_score(y_pred, f_y_test)}, accuracy_score reconstructions: {accuracy_score(y_pred_recon, f_y_test)}")

accuracy_score baseline: 0.8436, accuracy_score reconstructions: 0.8463


In [67]:
k_classifier = LogisticRegression(solver='newton-cg')
k_classifier.fit(k_x_train, k_y_train)

y_pred = k_classifier.predict(k_x_test)
y_pred_recon = k_classifier.predict(reconstructions_k_x_test)
print(f"accuracy_score baseline: {accuracy_score(y_pred, k_y_test)}, accuracy_score reconstructions: {accuracy_score(y_pred_recon, k_y_test)}")

accuracy_score baseline: 0.6943, accuracy_score reconstructions: 0.7074


In [63]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(x_train, y_train)

y_pred_rf = rf_classifier.predict(x_test)
y_pred_rf_recon = rf_classifier.predict(reconstructions_x_test)
print(f"accuracy_score baseline: {accuracy_score(y_pred_rf, y_test)}, accuracy_score reconstructions: {accuracy_score(y_pred_rf_recon, y_test)}")

accuracy_score baseline: 0.9683, accuracy_score reconstructions: 0.9654


In [68]:
f_rf_classifier = RandomForestClassifier()
f_rf_classifier.fit(f_x_train, f_y_train)
y_pred_rf = f_rf_classifier.predict(f_x_test)
y_pred_rf_recon = f_rf_classifier.predict(reconstructions_f_x_test)
print(f"accuracy_score baseline: {accuracy_score(y_pred_rf, f_y_test)}, accuracy_score reconstructions: {accuracy_score(y_pred_rf_recon, f_y_test)}")

accuracy_score baseline: 0.8777, accuracy_score reconstructions: 0.8482


In [69]:
k_rf_classifier = RandomForestClassifier()
k_rf_classifier.fit(k_x_train, k_y_train)
y_pred_rf = k_rf_classifier.predict(k_x_test)
y_pred_rf_recon = k_rf_classifier.predict(reconstructions_k_x_test)
print(f"accuracy_score baseline: {accuracy_score(y_pred_rf, k_y_test)}, accuracy_score reconstructions: {accuracy_score(y_pred_rf_recon, k_y_test)}")

accuracy_score baseline: 0.8538, accuracy_score reconstructions: 0.8226


### Wnioski:
Wszystkie wyniki rekonstrukcji zostały poprawione względem autoencodera z poprzedniego zadania.\
Dla klasyfikatora logistic regression wyniki zostały poprawione po zastosowaniu ekstrakcji cech względem baseline-u.\
Dla klasyfikatora random forest nie udało się poprawić wyników względem baseline-u, ale są one lepsze niż dla logistic regression.